# Modules

In [3]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori, association_rules

# Data

In [4]:
dataM= pd.read_csv("members.csv")

/tmp/ipykernel_311/1274091182.py:1: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  dataM= pd.read_csv("members.csv")


Marca un advertencia por tener valores mezclados, pero en temas de carga de los datos no se generá complejidad mayor

In [5]:
dataM.head()

,id,name,joined,topics,groupId,Unnamed: 5
0,173425062,Aaron,1.46854E+12,223;8652;9696;10099;10306;10454;10581;15236;15...,10027652,NaN
1,73726832,Aaron Elligsen,1.45067E+12,188;455;563;659;789;3833;7029;15582;19243;2150...,10027652,NaN
2,185309112,Aaron McGeever,1.4662E+12,85;317;3833;6960;15083;15360;18296;18864;20629...,10027652,NaN
3,185045966,Aaron Reisman,1.45258E+12,659;3833;9696;10209;16216;48471;108403;5938;78...,10027652,NaN
4,121471852,Aaron Strick,1.42792E+12,24099;52235;89730;608312;1432472;9696;10290;10579,10027652,NaN


In [6]:
dataM.drop(columns='Unnamed: 5',inplace=True)

In [7]:
dataG= pd.read_csv("groups.csv")

In [8]:
dataG.head()

,id,name,urlname,link,rating,created,description,organiserName,organiserMemberId
0,1174738,The San Francisco WordPress Meetup,wordpress-sf,http://www.meetup.com/wordpress-sf/,4.68,1.212370e+12,"<p>This meetup is all about the 'Press, from b...",Zach Berke,11611445.0
1,14080012,Bay Area Music Tech Meetup,Bay-Area-Music-Tech-Meetup,http://www.meetup.com/Bay-Area-Music-Tech-Meetup/,0.00,1.398340e+12,<p><span>Concentrating on the confluence of Mu...,Darren Majewski,63615542.0
2,19103758,Sports Hackathon,SportsHackathon,http://www.meetup.com/SportsHackathon/,0.00,1.447050e+12,<p><span>Sports Hackathon is a series of event...,Akif Malik,192033339.0
3,18960403,Incubation Station,Incubationstation,http://www.meetup.com/Incubationstation/,5.00,1.443020e+12,<p>Being an entrepreneur is tough... we all kn...,Josh Jermaine,14125597.0
4,10524692,Docker Palo Alto,Docker-Palo-Alto,http://www.meetup.com/Docker-Palo-Alto/,4.62,1.380740e+12,<p>Meet other developers and ops engineers usi...,Moby Dock,114766932.0


# Exploratory data analysis

## Missings

In [10]:
dataG.isna().mean()

id                   0.000000
name                 0.000000
urlname              0.000000
link                 0.000000
rating               0.000000
created              0.000000
description          0.000000
organiserName        0.001647
organiserMemberId    0.001647
dtype: float64

In [11]:
dataM.isna().mean()

id         0.000000
name       0.000019
joined     0.000004
topics     0.176539
groupId    0.000000
dtype: float64

Aparentemente hay datos en tolos los campos que indican el identificador de grupo, aunque más adelante observaremos que hay ciertas incosistencias en el mismo.

## Inconsistencies

Tenemos listas en groupId, además de filas duplicadas 

In [63]:
dataM[["id", 'groupId']].value_counts()

id         groupId                                                                                                                                                                                                                                                                                                           
255851     33135;38660;108403;288631;15401;15582;659;3833;9696;48471;128595;563;7029;17628;164111;15720;16390;10209;10102;1434182;87614;126177;166000;46810;2109;1454542;15167;28990;455;123243;128799;27954;132503;805252;46831;188;17742;20479;676752;123242;89943;89944;1273;32623;151695;32743;134870;124668;152026;15472    57
110411952  2065031                                                                                                                                                                                                                                                                                                                2
9069479    6487212                

In [66]:
dataM.drop_duplicates(subset=["id", "groupId"],inplace=True)

In [67]:
dataM[["id", 'groupId']].value_counts()

id         groupId 
17         18211855    1
150632942  2065031     1
150630132  15936192    1
150626892  9226282     1
           8567532     1
                      ..
36560042   1998291     1
36557112   844726      1
36554532   455214      1
36552752   844726      1
212492189  17747852    1
Length: 537908, dtype: int64

## Top 10 groups

In [68]:
top =dataM.groupby("groupId")[["id"]].count().copy()

### With more members

In [69]:
top[["id"]].sort_values(by= "id", ascending=False).head(10)

,id
groupId,
844726,27006
1619955,14875
2065031,11007
3483762,9860
13402242,9260
677520,8660
9226282,8578
5918982,8537
3138542,7850


### With least members

In [70]:
top[["id"]].sort_values(by= "id", ascending=True).head(10)

,id
groupId,
33135;38660;108403;288631;15401;15582;659;3833;9696;48471;128595;563;7029;17628;164111;15720;16390;10209;10102;1434182;87614;126177;166000;46810;2109;1454542;15167;28990;455;123243;128799;27954;132503;805252;46831;188;17742;20479;676752;123242;89943;89944;1273;32623;151695;32743;134870;124668;152026;15472,1
17069312,5
13833032,6
20394467,7
18693893,9
18570876,9
15009682,11
19875891,11
19276237,15


## Top users

In [71]:
topUser =dataM.groupby("id")[["groupId"]].count().copy()

### In more groups

In [72]:
topUser.sort_values(by="groupId",ascending=False).head()

,groupId
id,
159354732,336
2853561,302
3893698,256
11781658,223
5220940,221


### In a few groups

In [73]:
topUser.sort_values(by="groupId",ascending=True).head()

,groupId
id,
112794192,1
185685817,1
84578192,1
84578182,1
84573632,1


## Items per basket

In [74]:
topUser.describe()

,groupId
count,190329.000000
mean,2.826201
std,5.339689
min,1.000000
25%,1.000000
50%,1.000000
75%,3.000000
max,336.000000


## Number of groups

In [82]:
items =dataG.size
items

5463

## Number of users

In [83]:
baskets=topUser.size
baskets

190329

## Sparsity

Tomando en cuenta que tomamos cada grupo como una columna y cada usuario como fila, y que siendo optimistas cada usuario en promedio esta en 3 grupos cada uno, entonces tendriamos que

In [86]:
num_celdas= baskets*items
celdas_conValor = items*3
(num_celdas,celdas_conValor)

(1039767327, 16389)

In [87]:
sparsity = 1- (celdas_conValor/num_celdas)
sparsity

0.9999842378197752

Por lo cual tendríamos una matriz muy vacia lo cual dificultará el análisis de recomendaciones, procederemos a dar una implementación tomando pocos grupos, con el fin de hacerlo un poco más viable.

# Preprossesing

## Members with multiple groups

In [88]:
dataM=dataM.apply(lambda x: x if len(str(x[4]))<15 else None,axis=1)

## Group Missings

In [89]:
dataM.dropna(how='all',inplace=True)

In [90]:
dataM['groupId'] = dataM['groupId'].apply(lambda x: int(x))

In [91]:
dataM.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 537907 entries, 0 to 538032
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       537907 non-null  float64
 1   name     537897 non-null  object 
 2   joined   537905 non-null  object 
 3   topics   442937 non-null  object 
 4   groupId  537907 non-null  int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 24.6+ MB


In [92]:
dataG.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 607 entries, 0 to 606
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 607 non-null    int64  
 1   name               607 non-null    object 
 2   urlname            607 non-null    object 
 3   link               607 non-null    object 
 4   rating             607 non-null    float64
 5   created            607 non-null    float64
 6   description        607 non-null    object 
 7   organiserName      606 non-null    object 
 8   organiserMemberId  606 non-null    float64
dtypes: float64(3), int64(1), object(5)
memory usage: 42.8+ KB


## Merge

In [93]:
data=dataM.merge(right=dataG, left_on="groupId",
         right_on="id", how= 'inner')[['id_x', 'name_x','id_y', 'name_y']]

In [94]:
data=data.rename({'id_x':'member_id','name_x': 'member_name','id_y': 'group_id', 'name_y': 'group_name'}, axis=1)

In [95]:
data.size/dataM.size

0.8

In [96]:
len(data['group_id'].unique()),len(dataM["groupId"].unique())

(594, 594)

## Top Groups as columns

In [97]:
df=data.copy()

In [98]:
ls_groups=df['group_name'].value_counts().head(70).index

In [99]:
ls= [0 for x in range(len(df))]
for column in ls_groups:
    df.insert(1,column,ls)

In [100]:
df.head()

,member_id,Bay Area Women in Machine Learning & Data Science,DataKind SF Bay Area,Code Self Study,Elasticsearch San Francisco,Developing with Data in San Francisco,San Francisco Bay Area Big Data and Scalable Systems,Silicon Valley IoT and Wearable Devices,SFNode,Ember.js San Francisco,...,Silicon Valley Cloud Computing,SF Data Science,Docker Online Meetup,Tech in Motion Events: San Francisco,SF Data Mining,SFHTML5,Silicon Valley Entrepreneurs & Startups,member_name,group_id,group_name
0,173425062.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,Aaron,10027652,Elm user group SF
1,73726832.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,Aaron Elligsen,10027652,Elm user group SF
2,185309112.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,Aaron McGeever,10027652,Elm user group SF
3,185045966.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,Aaron Reisman,10027652,Elm user group SF
4,121471852.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,Aaron Strick,10027652,Elm user group SF


### Delete unpopular groups

In [101]:
ls= []
for index, row in df.iterrows():
    if(row["group_name"] not in ls_groups):
        ls.append(index)

In [102]:
len(ls)/len(df)

0.4365308501283679

In [103]:
df.drop(ls, axis=0, inplace=True)

### Encoder groups

In [104]:
def encoder(x):
    group = str(x["group_name"])
    x[group]=1
    return x

In [105]:
df=df.apply(lambda x: encoder(x),axis=1)

In [106]:
df_groups= df.groupby("member_id")[ls_groups].sum()

In [107]:
df_bool= df_groups.copy() >0

In [108]:
df_bool.sample(10)

,Silicon Valley Entrepreneurs & Startups,SFHTML5,SF Data Mining,Tech in Motion Events: San Francisco,Docker Online Meetup,SF Data Science,Silicon Valley Cloud Computing,The Hive Think Tank,Bay Area Spark Meetup,Data Visualization Group in the Bay Area,...,DataStax Cassandra SF Users,Ember.js San Francisco,SFNode,Silicon Valley IoT and Wearable Devices,San Francisco Bay Area Big Data and Scalable Systems,Developing with Data in San Francisco,Elasticsearch San Francisco,Code Self Study,DataKind SF Bay Area,Bay Area Women in Machine Learning & Data Science
member_id,,,,,,,,,,,,,,,,,,,,,
16042521.0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
31071232.0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
47871982.0,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
57493052.0,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
11779693.0,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
195502874.0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
188660433.0,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3550110.0,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
188658436.0,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


# Frequent groups

In [109]:
frequent_itemsets = apriori(df_bool, 
                            min_support =.01, 
                            max_len = 3, 
                            use_colnames = True) 

In [110]:
print(frequent_itemsets.shape)
frequent_itemsets.sort_values("support").tail()

(110, 2)


,support,itemsets
4,0.065538,(Docker Online Meetup)
3,0.069785,(Tech in Motion Events: San Francisco)
2,0.077902,(SF Data Mining)
1,0.105278,(SFHTML5)
0,0.191136,(Silicon Valley Entrepreneurs & Startups)


## Confidence

In [111]:
rules = association_rules(frequent_itemsets, 
                          metric = 'confidence', 
                          min_threshold =.4 )

In [112]:
print(rules.shape)
rules.head()

(21, 9)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(SF Founder Club),(Silicon Valley Entrepreneurs & Startups),0.037808,0.191136,0.018048,0.477349,2.497432,0.010821,1.547619
1,(Angular-SF),(SFHTML5),0.026067,0.105278,0.012060,0.462666,4.394692,0.009316,1.665113
2,(SF Data Science),(SF Data Mining),0.065503,0.077902,0.027595,0.421286,5.407860,0.022493,1.593356
3,(SF Bayarea Machine Learning),(SF Data Mining),0.039153,0.077902,0.018069,0.461497,5.924030,0.015019,1.712334
4,(SF Big Analytics),(SF Data Mining),0.036428,0.077902,0.014757,0.405090,5.199966,0.011919,1.549979


In [113]:
rules_data_consecuent = rules[rules['antecedents'] == {'SF Data Science'}]
rules_data_consecuent.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2,(SF Data Science),(SF Data Mining),0.065503,0.077902,0.027595,0.421286,5.40786,0.022493,1.593356


## Lift

In [114]:
rules = association_rules(frequent_itemsets, 
                          metric = 'lift', 
                          min_threshold =1 )

In [115]:
print(rules.shape)
rules.head()

(88, 9)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Silicon Valley Entrepreneurs & Startups),(Tech in Motion Events: San Francisco),0.191136,0.069785,0.015776,0.082537,1.182743,0.002437,1.013900
1,(Tech in Motion Events: San Francisco),(Silicon Valley Entrepreneurs & Startups),0.069785,0.191136,0.015776,0.226065,1.182743,0.002437,1.045131
2,(Silicon Valley Cloud Computing),(Silicon Valley Entrepreneurs & Startups),0.061292,0.191136,0.012754,0.208083,1.088665,0.001039,1.021400
3,(Silicon Valley Entrepreneurs & Startups),(Silicon Valley Cloud Computing),0.191136,0.061292,0.012754,0.066726,1.088665,0.001039,1.005823
4,(The Hive Think Tank),(Silicon Valley Entrepreneurs & Startups),0.060421,0.191136,0.012513,0.207099,1.083513,0.000964,1.020132


### Groups related to SF Data Science

In [116]:
rules_data_consecuent = rules[rules['antecedents'] == {'SF Data Science'}]
rules_data_consecuent.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
18,(SF Data Science),(SF Data Mining),0.065503,0.077902,0.027595,0.421286,5.407860,0.022493,1.593356
39,(SF Data Science),(Bay Area Spark Meetup),0.065503,0.055559,0.013306,0.203133,3.656195,0.009667,1.185194
41,(SF Data Science),(Data Visualization Group in the Bay Area),0.065503,0.051383,0.013808,0.210805,4.102625,0.010443,1.202006
42,(SF Data Science),(SF Bayarea Machine Learning),0.065503,0.039153,0.014573,0.222474,5.682184,0.012008,1.235775
45,(SF Data Science),(SF Big Analytics),0.065503,0.036428,0.016271,0.248406,6.819083,0.013885,1.282038
